<a href="https://colab.research.google.com/github/bdemchak/cytoscape-jupyter/blob/main/echo/linux-3_10-killer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This test is designed to kill Cytoscape v3.10.0-SNAPSHOT on Linux Ubuntu 22.04.1 LTS. It does not kill Cytoscape on a Windows system, and v3.9.1 doesn't die on either Windows or Ubuntu.

The **Network Load test** sends a series of network import and table import messages. This stresses Cytoscape and shows the client's ability to wait a long time for a reply.



---
# Setup data files, py4cytoscape and Cytoscape connection
**NOTE: To run this notebook, you must manually start Cytoscape first -- don't proceed until you have started Cytoscape.**

## Setup: Fetch latest py4cytoscape




**Note that you can fetch from the latest Github unreleased version by setting _PY4CYTOSCAPE to 'git+https://github.com/cytoscape/py4cytoscape' immediately before the exec() call. To fetch a particular branch, add '@' to the end (e.g., 'git+https://github.com/cytoscape/py4cytoscape@0.0.12').**

To load the default (PyPI) py4cytoscape version, do not set _PY4CYTOSCAPE at all.

In [ ]:
_PY4CYTOSCAPE = 'git+https://github.com/cytoscape/py4cytoscape@1.6.0' # optional
import requests

exec(requests.get("https://raw.githubusercontent.com/cytoscape/jupyter-bridge/master/client/p4c_init.py").text)

IPython.display.Javascript(_PY4CYTOSCAPE_BROWSER_CLIENT_JS) # Start browser client

## Setup: Sanity test to verify Cytoscape connection

By now, the connection to Cytoscape should be up and available. To verify this, try a simple operation that doesn't alter the state of Cytoscape.

In [ ]:
p4c.cytoscape_version_info()


## Setup: Initialize a basic static message
The basic message is a random digit sequence that is unlikely to be used by any other Colab Notebook instance.

In [ ]:
import uuid as uu
my_uuid = str(p4c.get_browser_client_channel())

print(f'Fixed message: {my_uuid}')


## Setup: Prepare Network Load and Column Tests

Load a .mitab network file and .soft attribute file into the Cytoscape sandbox so Cytoscape can read them directly.

Also, define the load_network() function, which is where the trouble actually happens during the test.

In [ ]:
res_mitab = p4c.sandbox_url_to("https://www.dropbox.com/s/8wc8o897tsxewt1/BIOGRID-ORGANISM-Saccharomyces_cerevisiea-3.2.105.mitab?dl=0", "BIOGRID-ORGANISM-Saccharomyces_cerevisiea-3.2.105.mitab")
res_soft = p4c.sandbox_url_to("https://www.dropbox.com/s/r15azh0xb53smu1/GDS112_full.soft?dl=0", "GDS112_full.soft")

def load_network():
  print(' closing session')
  p4c.close_session(False)
  try:
    print(' importing file')
    p4c.import_network_from_file("BIOGRID-ORGANISM-Saccharomyces_cerevisiea-3.2.105.mitab")
  except:
    pass
  print(' importing table')
  p4c.load_table_data_from_file("GDS112_full.soft", start_load_row=83, data_key_column_index=10, delimiters="\t")

## Network Load Test
Load a network and then load the node table data. Both are substantial files that were already staged on the Cytoscape sandbox in a previous step.

This is where death occurs. Most often, Cytoscape just disappears. Sometimes it grabs all memory and then ignores all input. Other times, it can kill the browswer, too.

Given this, I suspect the problem is with the JVM.

In [ ]:
%%time

for i in range(0, 10):
  load_network()
  print(f'Load network {i}: {my_uuid} successful')